<center><h1> Build and track our portfolio based on our trades.<h1></center>

### Doesn't have any concept of buy and sell just yet.  only that a position is bought and held

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
import pylab
import requests as rq
import matplotlib
from helper_scripts import *
from IPython.display import HTML
from datetime import datetime
from ipywidgets import interact, interact_manual

## We load the data into a pandas dataframe and Render as a nicer table

In [ ]:
trade_data = pd.read_csv('data/trades.csv', delimiter=',')
h = get_table_html(trade_data)
HTML(h)

In [ ]:
quotes = get_quotes(trade_data)

<!-- <div class="alert alert-danger" role="alert">
  ^^^ We  can see a few missing. Need to fix these later. 
</div> -->

In [ ]:
@interact()
def make_plot(ticker=trade_data['tickers']):

    ii = list(trade_data['tickers']).index(ticker)
    candlestick = go.Candlestick(x=quotes[ii].index,
                                open=quotes[ii]['Open'],
                                high=quotes[ii]['High'],
                                low=quotes[ii]['Low'],
                                close=quotes[ii]['Close'])

    fig = go.Figure(data=[candlestick])
    fig.update_layout(xaxis_rangeslider_visible=False,
                     title=trade_data['tickers'][ii],
                     template='plotly_dark',
                     width=1600, height=800)

    fig.show()

## Now we need to generate our port folio based on the ticker data.  Dealing with datetimes is always painful

In [ ]:
portfolio = generate_portfolio(trade_data, quotes)
portfolio.describe()

## Now we need to find our non-aud equities and convert to AUD

In [ ]:
cl =trade_data['currency'].unique()  #  TODO We are converting AUD to AUD with this.  very innefficient.  but breaks if we don't do it.
# cl = cl[cl!='AUD']  # if we're in AUD don't try and convert

for currency in cl:
    cols = trade_data[trade_data['currency']==currency]['tickers']
    for ii, item in enumerate(cols.values):
        c = convert_to_aud(portfolio[item], currency)
#         print(c)
        portfolio[item] = c

In [ ]:
portfolio.head()

In [ ]:
def write_to_excel(file='data/converted_portfolio.xls'):
    portfolio.to_excel(file)
write_to_excel()

In [ ]:
import plotly.express as px

fig = px.line(portfolio, width=1600, height=800)
fig.update_layout(xaxis_rangeslider_visible=False,
                 title='Portfolio Value',
                 template='plotly_dark')
fig.show()

In [ ]:
def make_pie(df):
    pylab.pie(df, labels=df.index, shadow = True, autopct='%1.1f%%', textprops={'color':"w"})

In [ ]:
make_pie(trade_data.groupby(['market'] )['tickers'].count())
pylab.title('Market', c='WHITE')

In [ ]:
@interact()
def plot_pie(day=(0, portfolio.count()[0]-1)):
    tmp = portfolio.copy()
    tmp = tmp.drop('Total', axis=1)
    vec = tmp.iloc[day].to_numpy()[:-1]
    pylab.pie(vec, labels=tmp.columns[:-1], shadow = True, autopct='%1.1f%%', textprops={'color':"w"});
    fig = matplotlib.pyplot.gcf()
    fig.set_size_inches(18*2, 10*2)

In [ ]:
import plotly.express as px

portfolio.corr();

fig = px.imshow(portfolio.corr(), width=1000, height=1000)
fig.update_layout(
    title= 'Pearson Correlation Coefficient')
fig.show()